In [75]:
import pandas as pd
import numpy as np

filename = "./data/Exported/DifferencePichertsStatsTeam22.csv"
df = pd.read_csv(filename)

def pivot_var(df, category1, category2, value):
    df_last = df.groupby(category1)[category2].last().reset_index()
    df_first = df.groupby(category1)[category2].first().reset_index()

    df_previous = df.merge(df_first, how='left', on=[category1, category2])
    df_new = df.merge(df_last, how='right', on=[category1, category2])
    df_new[category2] = 'Actual' + value

    frames = [df, df_last]
    df_prev = pd.concat(frames)
    df_prev.drop_duplicates(subset=[category1, category2], keep=False, inplace=True)
    df_prev[category2] = 'Previous' + value

    frames = [df_prev, df_new]
    df_fixed = pd.concat(frames)
    df_fixed.sort_values(by=[category1], inplace=True)
    piv_table = pd.pivot_table(df_fixed, values=value, index=[category1],
                        columns=[category2], aggfunc=np.mean)
    piv_table = piv_table.reset_index()
    return piv_table

piv_table = pivot_var(df, 'Name', 'Team', 'CSW%')
piv_table.to_csv("./data/Exported/PowerBICSW%.csv", index=False)